In [1]:
!pip install -q pinecone-client
!pip install -U -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.8 MB/s eta 0:00:00


In [4]:
!pip install -q jsonlines

In [28]:
from pinecone import Pinecone

pc = Pinecone(api_key="51bd3ca8-ae20-4999-bacc-7fa2ce57ff4f")
index = pc.Index("chatindex")

In [29]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd drive/MyDrive

/content/drive/MyDrive


In [8]:
import json, jsonlines
import pandas as pd

def load_and_clean_json(file_path):
    try:
        with jsonlines.open(file_path, 'r') as reader:
            # Read each line separately
            lines = [line for line in reader]

            return lines

    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
file_path = 'DATA_final.json'
DATA = load_and_clean_json(file_path)
DATA

[{'amende': 'le dépassement non réglementaire ',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement illégal',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement horslaloi',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non autorisé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non permis',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non encadré',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non conforme',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non contrôlé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non surveillé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non conventionnel',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépassement non approuvé',
  'feature_class': ['400 DH', '500 DH', 4]},
 {'amende': ' le dépa

In [10]:
# first we need the encodings
import torch
corpus_embeddings = torch.load("corpus_embeddings.pth")

In [12]:
corpus_embeddings.shape

torch.Size([2694, 768])

In [17]:
str(['400 DH', '500 DH', 4])

"['400 DH', '500 DH', 4]"

In [34]:
def batch_insert(batch_size=100):
  batch=0
  vectors=[]
  i=0
  for data, encoding in zip(DATA,corpus_embeddings) :
    # print(data, encoding)
    # break
    i+=1
    vector={"id":f'id{i}','values':'', "metadata":{"class_violation":''}}
    vector['metadata']['class_violation']=str(data['feature_class'])
    vector['values']=encoding
    vectors.append(vector)
    batch+=1
    if batch>=batch_size:
      index.upsert(vectors=vectors)
      vectors=[]
      batch=0
  # Insert any remaining vectors if the total is not a perfect multiple of batch_size
  if vectors:
      index.upsert(vectors=vectors)

In [35]:
batch_insert()

In [37]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.02694,
 'namespaces': {'': {'vector_count': 2694}},
 'total_vector_count': 2694}

In [ ]:
normal_values = [tensor.item() for tensor in corpus_embeddings[0]]
normal_values

In [49]:
result = index.query(vector=normal_values, top_k=1)
result

{'matches': [{'id': 'id1', 'score': 1.0006597, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}